In [11]:
!pip install --upgrade keras

In [1]:
import tensorflow as tf

In [2]:
print(tf.__version__)

2.11.0


In [3]:
from paho.mqtt import client as mqtt
import json
import random
import time
import numpy as np
from keras.models import load_model
import pandas as pd
import pickle

In [6]:
#reading the test dataset file, define the path to test dataset
df = pd.read_csv(r"C:\Users\HP\Downloads\Oct-2020_TestData.csv")
df

,datetime,temp,dew,humidity,sealevelpressure,winddir,solarradiation,windspeed,preciptype
0,2020-09-11,28.9,25.9,84.4,1002.7,141.5,110.2,16.6,1
1,2020-09-12,28.2,25.4,85.2,1004.1,147.0,197.4,27.0,1
2,2020-09-13,28.3,25.3,84.4,1006.6,196.9,213.1,22.3,1
3,2020-09-14,27.6,25.1,86.8,1006.3,165.5,156.1,19.6,1
4,2020-09-15,27.7,25.2,86.5,1006.2,153.1,131.6,16.9,1
5,2020-09-16,27.7,25.7,88.9,1006.4,143.0,130.6,15.9,1
6,2020-09-17,27.3,25.5,90.2,1006.9,163.2,143.2,16.6,1
7,2020-09-18,27.0,25.3,90.4,1005.9,132.1,99.7,9.4,1
8,2020-09-19,27.4,26.1,92.7,1003.6,151.6,178.7,10.0,1
9,2020-09-20,28.9,25.4,82.0,1001.8,171.3,80.5,17.2,1


In [7]:
df["datetime"] = pd.to_datetime(df["datetime"])
df.set_index("datetime", inplace = True)
df['seconds'] = df.index.map(pd.Timestamp.timestamp)
week_sec = 60*60*24*7
month_sec = 60*60*24*30.4375
df.insert(7,'Week_Sin', np.sin(2*np.pi*df['seconds']/week_sec))
df.insert(8,'Week_Cos', np.cos(2*np.pi*df['seconds']/week_sec))
df.insert(9,'Month_Sin', np.sin(2*np.pi*df['seconds']/month_sec))
df.insert(10,'Month_Cos', np.cos(2*np.pi*df['seconds']/month_sec))
df = df.drop('seconds',axis=1)
df.head()

,temp,dew,humidity,sealevelpressure,winddir,solarradiation,windspeed,Week_Sin,Week_Cos,Month_Sin,Month_Cos,preciptype
datetime,,,,,,,,,,,,
2020-09-11,28.9,25.9,84.4,1002.7,141.5,110.2,16.6,0.781831,0.623490,0.880683,-0.473706,1
2020-09-12,28.2,25.4,85.2,1004.1,147.0,197.4,27.0,0.974928,-0.222521,0.764891,-0.644159,1
2020-09-13,28.3,25.3,84.4,1006.6,196.9,213.1,22.3,0.433884,-0.900969,0.616621,-0.787260,1
2020-09-14,27.6,25.1,86.8,1006.3,165.5,156.1,19.6,-0.433884,-0.900969,0.442168,-0.896932,1
2020-09-15,27.7,25.2,86.5,1006.2,153.1,131.6,16.9,-0.974928,-0.222521,0.248940,-0.968519,1


In [9]:
#reading the dataset scaler module file with trained weights to scale data
with open(r"C:\Users\HP\Downloads\StandardScaler.pkl", 'rb') as f:
  scaler = pickle.load(f)

X = scaler.transform(df.iloc[:, :-1])

2024-04-23 15:55:27,626 [31376] WARNING  py.warnings:109: [JupyterRequire] C:\Users\HP\anaconda3\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.2.0 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(



In [10]:
X

array([[ 2.84483991e-01,  9.39134933e-01,  1.02094310e+00,
        -1.66743629e+00, -1.27074570e+00, -2.06895041e+00,
        -8.97937837e-01,  1.10378393e+00,  8.82268685e-01,
         1.23804758e+00, -6.71110573e-01],
       [-7.27543164e-02,  8.28778957e-01,  1.07600263e+00,
        -1.28958045e+00, -1.15387943e+00, -5.39631617e-01,
         7.37569405e-01,  1.37682561e+00, -3.14339247e-01,
         1.07427121e+00, -9.12141421e-01],
       [-2.17202725e-02,  8.06707761e-01,  1.02094310e+00,
        -6.14837866e-01, -9.35836845e-02, -2.64284082e-01,
        -1.55406035e-03,  6.11779831e-01, -1.27394430e+00,
         8.64556694e-01, -1.11449458e+00],
       [-3.78958580e-01,  7.62565371e-01,  1.18612169e+00,
        -6.95806975e-01, -7.60783815e-01, -1.26395348e+00,
        -4.26156902e-01, -6.15258334e-01, -1.27394430e+00,
         6.17808896e-01, -1.26957777e+00],
       [-3.27924536e-01,  7.84636566e-01,  1.16547437e+00,
        -7.22796679e-01, -1.02426412e+00, -1.69363594e+00,
  

In [11]:
def create_sequences(x, y, time_steps=20):
    xseq, yseq = [], []
    for i in range(len(x) - time_steps):
        xseq.append(x[i:i+time_steps])
        yseq.append(y[i+time_steps])
    return np.array(xseq), np.array(yseq)

In [19]:
y = df.iloc[:,-1]
data_seq = create_sequences(X, y, time_steps=20)
data_seq[0][0]

array([[ 2.84483991e-01,  9.39134933e-01,  1.02094310e+00,
        -1.66743629e+00, -1.27074570e+00, -2.06895041e+00,
        -8.97937837e-01,  1.10378393e+00,  8.82268685e-01,
         1.23804758e+00, -6.71110573e-01],
       [-7.27543164e-02,  8.28778957e-01,  1.07600263e+00,
        -1.28958045e+00, -1.15387943e+00, -5.39631617e-01,
         7.37569405e-01,  1.37682561e+00, -3.14339247e-01,
         1.07427121e+00, -9.12141421e-01],
       [-2.17202725e-02,  8.06707761e-01,  1.02094310e+00,
        -6.14837866e-01, -9.35836845e-02, -2.64284082e-01,
        -1.55406035e-03,  6.11779831e-01, -1.27394430e+00,
         8.64556694e-01, -1.11449458e+00],
       [-3.78958580e-01,  7.62565371e-01,  1.18612169e+00,
        -6.95806975e-01, -7.60783815e-01, -1.26395348e+00,
        -4.26156902e-01, -6.15258334e-01, -1.27394430e+00,
         6.17808896e-01, -1.26957777e+00],
       [-3.27924536e-01,  7.84636566e-01,  1.16547437e+00,
        -7.22796679e-01, -1.02426412e+00, -1.69363594e+00,
  

In [14]:
#reading the trainable model weights file (importing the trained model)
model = load_model(r"C:\Users\HP\Downloads\LSTMModel_95.5.h5")
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (None, 20, 128)          38912     
 nal)                                                            
                                                                 
 dropout_7 (Dropout)         (None, 20, 128)           0         
                                                                 
 lstm_8 (LSTM)               (None, 20, 64)            49408     
                                                                 
 dropout_8 (Dropout)         (None, 20, 64)            0         
                                                                 
 lstm_9 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_9 (Dropout)         (None, 32)                0         
                                                      

In [15]:
pred = model.predict(data_seq[0])
pred

1/1 [==============================] - 1s 1s/step


array([[0.9927996 ],
       [0.9974153 ],
       [0.9969542 ],
       [0.9313981 ],
       [0.87531275],
       [0.8392169 ],
       [0.89533055],
       [0.9360406 ],
       [0.9966818 ],
       [0.9974277 ],
       [0.99756277],
       [0.99751645],
       [0.9742562 ],
       [0.96242434],
       [0.9779036 ],
       [0.99761057],
       [0.9974485 ],
       [0.8207223 ],
       [0.81189597],
       [0.8051599 ],
       [0.78914607],
       [0.83850944],
       [0.906435  ],
       [0.87933326],
       [0.82067484],
       [0.7365948 ],
       [0.5920658 ],
       [0.00705417],
       [0.00461539],
       [0.00357037],
       [0.01499489]], dtype=float32)

In [44]:
#rawpath=r"C:\Users\HP\Downloads\Oct-2020_TestData.csv"
#df2=pd.read_csv(rawpath)
#df2.drop(columns=['datetime','preciptype'],inplace=True)

In [56]:
raw = df.iloc[:,:-5].values.astype('float32')

array([[  28.9,   25.9,   84.4, 1002.7,  141.5,  110.2,   16.6],
       [  28.2,   25.4,   85.2, 1004.1,  147. ,  197.4,   27. ],
       [  28.3,   25.3,   84.4, 1006.6,  196.9,  213.1,   22.3],
       [  27.6,   25.1,   86.8, 1006.3,  165.5,  156.1,   19.6],
       [  27.7,   25.2,   86.5, 1006.2,  153.1,  131.6,   16.9],
       [  27.7,   25.7,   88.9, 1006.4,  143. ,  130.6,   15.9],
       [  27.3,   25.5,   90.2, 1006.9,  163.2,  143.2,   16.6],
       [  27. ,   25.3,   90.4, 1005.9,  132.1,   99.7,    9.4],
       [  27.4,   26.1,   92.7, 1003.6,  151.6,  178.7,   10. ],
       [  28.9,   25.4,   82. , 1001.8,  171.3,   80.5,   17.2],
       [  28.1,   26.3,   90.4, 1000.7,  115.1,  147.4,   17.3],
       [  26.7,   25.8,   95. , 1001.1,  170.2,   77.1,   18.4],
       [  26. ,   25.7,   98.2, 1002.9,  236.5,  148.7,   21.7],
       [  27.4,   25.8,   91.6, 1006.9,  114.2,  170.1,   16.1],
       [  28. ,   25.6,   86.9, 1007.7,  200. ,  177.4,   16.6],
       [  29. ,   24.2,  

In [61]:

# Define MQTT parameters
MQTT_BROKER = "localhost"
MQTT_PORT = 1883  # Default MQTT port
MQTT_TOPIC = "Incoming data stream"

# Initialize MQTT client
mqtt_client = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION2)
mqtt_client.connect(MQTT_BROKER, MQTT_PORT, 60)


# Callback
def on_publish(client, userdata, mid, reason_code, properties):
    # reason_code and properties will only be present in MQTTv5. It's always unset in MQTTv3
    try:
        # userdata.remove(mid)
        print("data published")
    except KeyError:
        print("on_publish() is called with a mid not present in unacked_publish")
        print("This is due to an unavoidable race-condition:")
        print("* publish() return the mid of the message sent.")
        print("* mid from publish() is added to unacked_publish by the main thread")
        print("* on_publish() is called by the loop_start thread")
        print("While unlikely (because on_publish() will be called after a network round-trip),")
        print(" this is a race-condition that COULD happen")
        print("")
        print("The best solution to avoid race-condition is using the msg_info from publish()")
        print("We could also try using a list of acknowledged mid rather than removing from pending list,")
        print("but remember that mid could be re-used !")


mqtt_client.on_publish = on_publish
avgt=[]
avgd=[]
avgh=[]
avgw=[]

def get_data(i):
    d = np.ndarray.tolist(raw[i])
    _pred = pred.tolist()
    avgt.append(d[0])
    avgd.append(d[1])
    avgh.append(d[2])
    avgw.append(d[6])
    return {
        "temp": d[0],
        "avgt": np.mean(avgt),
        "dew": d[1],
        "avgd": np.mean(avgd),
        "humidity": d[2],
        "avgh": np.mean(avgh),
        "sealevelpressure": d[3],
        "winddir": d[4],
        "solarradiation": d[5],
        "windspeed": d[6],
        "avgw": np.mean(avgw),
        "model_out": _pred[i],
    }


def publish_random_data():
    i =0 
    while True:
        json_data = get_data(i)        
        json_data = json.dumps(json_data)
        #print(json_data)
        mqtt_client.publish(MQTT_TOPIC, json_data)
        time.sleep(10)  # Publish data every 1 second
        i+=1
        if i == len(pred):
          i = 0


if __name__ == "__main__":
    try:
        publish_random_data()
    except KeyboardInterrupt:
        print("Program stopped by user.")

data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
Program stopped by user.
